In [0]:
file = open('Stephen King.txt')
story = file.read()
file.close()
story = story.lower()

In [0]:
vocab = sorted(set(story))

In [0]:
vocab_to_num = dict((j,i) for i,j in enumerate(vocab))
num_to_vocab = dict((i,j) for i,j in enumerate(vocab))

In [0]:
seq_len = 100
sequence = []
target = []
for i in range(len(story)-seq_len-1):
  sequence.append(story[i:i+seq_len])
  target.append(story[i+seq_len])

In [0]:
sequence_encode = []
target_encode = []
for i in sequence:
  temp = []
  for j in i:
    temp.append(vocab_to_num[j])
  sequence_encode.append(temp)
for i in target:
  target_encode.append(vocab_to_num[i])

In [0]:
import numpy as np

sequence_encode = np.array(sequence_encode)
target_encode = np.array(target_encode)

In [0]:
print(sequence_encode.shape, target_encode.shape)

(122769, 100) (122769,)


In [0]:
from keras.models import Sequential
from keras.layers import CuDNNLSTM, Embedding, Dense, Dropout
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


In [0]:
model = Sequential()
model.add(Embedding(len(vocab), 10, input_length=seq_len, trainable=True))
model.add(CuDNNLSTM(512, return_sequences=True))
model.add(Dropout(0.2))
model.add(CuDNNLSTM(512, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(len(vocab), activation= 'softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
ck = ModelCheckpoint('/content/drive/My Drive/stephen_king2.hdf5', monitor='loss', verbose=1, save_best_only=True, mode='auto')
model.summary()





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 10)           520       
_________________________________________________________________
cu_dnnlstm_1 (CuDNNLSTM)     (None, 100, 512)          1073152   
_________________________________________________________________
dropout_1 (Dropout)          (None, 100, 512)          0         
_________________________________________________________________
cu_dnnlstm_2 (CuDNNLSTM)     (None, 512)               2101248   
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 52)                26676     

In [0]:
model.fit(sequence_encode, target_encode, batch_size=64, epochs=40, callbacks=[ck], verbose=1)

Epoch 1/40
122769/122769 [==============================] - 92s 746us/step - loss: 2.4894 - acc: 0.2863

Epoch 00001: loss improved from inf to 2.48939, saving model to /content/drive/My Drive/stephen_king2.hdf5
Epoch 2/40
122769/122769 [==============================] - 85s 690us/step - loss: 1.9357 - acc: 0.4281

Epoch 00002: loss improved from 2.48939 to 1.93574, saving model to /content/drive/My Drive/stephen_king2.hdf5
Epoch 3/40
122769/122769 [==============================] - 85s 689us/step - loss: 1.6655 - acc: 0.5004

Epoch 00003: loss improved from 1.93574 to 1.66552, saving model to /content/drive/My Drive/stephen_king2.hdf5
Epoch 4/40
122769/122769 [==============================] - 84s 688us/step - loss: 1.4872 - acc: 0.5484

Epoch 00004: loss improved from 1.66552 to 1.48718, saving model to /content/drive/My Drive/stephen_king2.hdf5
Epoch 5/40
122769/122769 [==============================] - 85s 690us/step - loss: 1.3461 - acc: 0.5865

Epoch 00005: loss improved from 1.4

In [0]:
model.load_weights('/content/drive/My Drive/stephen_king2.hdf5')

In [0]:
def sample(preds, temperature=0.7):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [0]:
seed = 'people clustered in small groups that had a tendency to break up and re-form with surprising speed. '
sent = ''
seed_encode = []
text = ''
for i in seed:
  seed_encode.append(vocab_to_num[i])
seed_encode = np.array(seed_encode)
for i in range(500):
  text = seed_encode[-seq_len:]
  text = np.expand_dims(text,axis=0)
  pred = model.predict(text)
  pred = sample(pred[0])
  seed_encode = np.append(seed_encode, pred)

In [0]:
message = ''
for i in seed_encode:
  message += num_to_vocab[i]

In [0]:
print(message)

people clustered in small groups that had a tendency to break up and re-form with surprising speed. looking out to be an undergrad named donald morris work and i passed all the look and looked at him. he was supposed to stay wiping across the water,
but they were working and shittered back of its an
appla night. the corpse in the grinder, expecting the crap outs.

the counted in the second. jagged sallow sound over the way back on the way - at least aloud.

i suspect there,' hall said softly. he was supposed to be our life counting fingers. renshaw sat on the thud that had a sudden goot of a n
